In [1]:
import pandas as pd
import numpy as np
import json
import os
import networkx as nx
#import pickle5 as pickle

from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import SpectralClustering, KMeans
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.preprocessing import LabelEncoder

from myclass.CleanMergeDataset import Clean_Merge_Dataset
from myclass.BonferroniTtest import Bonferroni_Ttest

In [2]:
if os.path.exists('final_dataset_common.json') is False:

    data_normal = pd.read_pickle('./data-ready/RNA_dataframe_normal').replace('/', '\\')
    data_tumor = pd.read_pickle('./data-ready/RNA_dataframe').replace('/', '\\')
    dataset_RNA, y_RNA, cases_id_RNA = Clean_Merge_Dataset(name='RNA').transform(data_normal, data_tumor)
    df_RNA = pd.concat([dataset_RNA, cases_id_RNA], axis=1)

    data_normal = pd.read_pickle('./data-ready/miRNA_dataframe_normal').replace('/', '\\')
    data_tumor = pd.read_pickle('./data-ready/miRNA_dataframe').replace('/', '\\')
    dataset_miRNA, y_miRNA, cases_id_miRNA= Clean_Merge_Dataset(name='miRNA').transform(data_normal, data_tumor)
    df_miRNA = pd.concat([dataset_miRNA, cases_id_miRNA], axis=1)

    data_normal = pd.read_pickle('./data-ready/illumina-27-450-normal').replace('/', '\\')
    data_tumor = pd.read_pickle('./data-ready/illumina450-27-tumor').replace('/', '\\')
    dataset_illumina, y_illumina, cases_id_illumina= Clean_Merge_Dataset(name='illumina').transform(data_normal, data_tumor)
    df_illumina = pd.concat([dataset_illumina, cases_id_illumina], axis=1)

    dataset_RNA = Bonferroni_Ttest(label_case_id_into_X=True, alpha=0.05).fit_transform(pd.concat([df_RNA, y_RNA], axis=1), y_RNA)
    dataset_miRNA = Bonferroni_Ttest(label_case_id_into_X=True, alpha=0.05).fit_transform(pd.concat([df_miRNA, y_miRNA], axis=1), y_miRNA)
    dataset_illumina = Bonferroni_Ttest(label_case_id_into_X=True, alpha=0.05).fit_transform(pd.concat([df_illumina, y_illumina], axis=1), y_illumina)

    cases_id = set(dataset_illumina['case_id']) & set(dataset_miRNA['case_id']) & set(dataset_RNA['case_id'])
    df_final_illumina = dataset_illumina.loc[dataset_illumina['case_id'].isin(cases_id)]
    df_final_rna = dataset_RNA.loc[dataset_RNA['case_id'].isin(cases_id)]
    df_final_mirna = dataset_miRNA.loc[dataset_miRNA['case_id'].isin(cases_id)]

    print(df_final_illumina.shape)
    print(df_final_rna.shape)
    print(df_final_mirna.shape)
    
    df_final_illumina.to_pickle('illumina_pickle.pkl')
    df_final_rna.to_pickle('rna_pickle.pkl')
    df_final_mirna.to_pickle('miRNA_pickle.pkl')
    
    my_dict = {
        'miRNA': df_final_mirna.to_dict(),
        'RNA': df_final_rna.to_dict(),
        'illumina': df_final_illumina.to_dict()
    }
    with open('final_dataset_common.json', 'w') as outfile:
        json.dump(my_dict, outfile)
    
    df_illumina = df_final_illumina.copy()
    df_mirna = df_final_mirna.copy()
    df_rna = df_final_rna.copy()
    
    del my_dict
    del df_final_illumina
    del df_final_rna
    del df_final_mirna
    del dataset_illumina
    del dataset_RNA
    del dataset_miRNA
else:
    df_illumina = pd.read_pickle('illumina_pickle.pkl')
    df_mirna = pd.read_pickle('miRNA_pickle.pkl')
    df_rna = pd.read_pickle('rna_pickle.pkl')
    #with open('illumina_pickle.pkl' ,'rb') as f:
        #df_illumina = pickle.load(f)
    #with open('mirna_pickle.pkl' ,'rb') as f:
        #df_mirna = pickle.load(f)
    #with open('rna_pickle.pkl' ,'rb') as f:
        #df_rna = pickle.load(f)

**Selecting atlas genes**

In [3]:
with open('all_ensgs.json') as f:
    genes = json.load(f)
print(df_rna.shape)
columns = [gene for gene in genes if gene in df_rna.columns]
columns.append('case_id')
columns.append('label')
df_rna = df_rna.loc[:, columns]

(508, 12965)


In [4]:
import numpy as np
import pandas as pd
import os

from scipy.spatial.distance import pdist, squareform, cdist
from copy import deepcopy
from sklearn.preprocessing import StandardScaler

class SimilarityNetworkFusion:
    def __init__(self, df_mirna, df_rna, df_illumina, k=3, mu=0.3):
        
        self.cases_id = df_rna.loc[:, 'case_id']
        self.rna = df_rna.copy()
        self.mirna = df_mirna.copy()
        self.illumina = df_illumina.copy()
        
        self.k = k
        self.mu = mu
        self.check_columns()
    
    def calculate_matrix(self):
        self.dict_dist = self.calculate_sim_matrix()
        if hasattr(self, 'w_rna') is False:
            self.w_rna = self.__weights__(self.rna, 'RNA', save_matrix=True)
            self.w_mirna = self.__weights__(self.mirna, 'miRNA', save_matrix=True)
            self.w_illumina = self.__weights__(self.illumina, 'Illumina', save_matrix=True)
        
        if hasattr(self, 'p_rna') is False:
            self.starting_p_rna = self.P_matrix(self.w_rna.to_numpy().tolist(), self.cases_id.shape[0], 'RNA', save_matrix=True)
            self.starting_p_mirna = self.P_matrix(self.w_mirna.to_numpy().tolist(), self.cases_id.shape[0], 'miRNA', save_matrix=True)
            self.starting_p_illumina = self.P_matrix(self.w_illumina.to_numpy().tolist(), self.cases_id.shape[0], 'Illumina', save_matrix=True)
                
        self.s_rna = self.S_matrix(self.w_rna.to_numpy().tolist(), self.cases_id.shape[0], 'RNA')
        self.s_mirna = self.S_matrix(self.w_mirna.to_numpy().tolist(), self.cases_id.shape[0], 'miRNA')
        self.s_illumina = self.S_matrix(self.w_illumina.to_numpy().tolist(), self.cases_id.shape[0], 'Illumina')
        
        return self
    
    def calculate_sim_matrix(self):
        tot = 0
        distance = 'euclidean'
        tot += pdist(self.rna, distance)
        tot += pdist(self.mirna, distance)
        tot += pdist(self.illumina, distance)
        tot = tot/3 
        df_dist = pd.DataFrame(columns=self.cases_id, index=self.cases_id, data=squareform(tot))
        
        return df_dist

    def __weights__(self, dataset, name, save_matrix=False):
        
        if 'weights_matrix_'+name+'.pkl' in os.listdir('.'):
            print('Read file pickle for weights matrix of {}'.format(name))
            weights = pd.read_pickle('weights_matrix_'+name+'.pkl')
            #dist = pdist(dataset, 'euclidean')
            #df_dist = pd.DataFrame(columns=self.cases_id, index=self.cases_id, data=squareform(dist))
            #self.dict_dist[name] = df_dist.copy()
            return weights
        
        print('Calculating weights for {}...'.format(name))
        df = pd.DataFrame(columns=self.cases_id, data=dataset.T.values)
        
        #calculate euclidean distance
        dist = pdist(dataset, 'euclidean')
        df_dist = pd.DataFrame(columns=self.cases_id, index=self.cases_id, data=squareform(dist))
        weights = pd.DataFrame(columns=self.cases_id, index=self.cases_id, data=[])
                
        for i, patient_i in enumerate(tqdm(self.cases_id)):
            for patient_j in self.cases_id.iloc[i:]:
                    tokK_mean_i = np.sort(df_dist.loc[patient_i, :].to_numpy())[:self.k].mean()
                    topK_mean_j = np.sort(df_dist.loc[patient_j, :].to_numpy())[:self.k].mean()
                    
                    eps = (tokK_mean_i + tokK_mean_i + df_dist.loc[patient_i, patient_j])/3

                    weights.loc[patient_i, patient_j] = np.exp(-(df_dist.loc[patient_i, patient_j]**2/(eps*self.mu)))
                    weights.loc[patient_j, patient_i] = np.exp(-(df_dist.loc[patient_j, patient_i]**2/(eps*self.mu)))
        if save_matrix:
            weights.to_pickle('weights_matrix_'+name+'.pkl')
        #self.dict_dist[name] = df_dist.copy()
        return weights       
    
    def check_columns(self):
        scaler = MinMaxScaler()
        if 'label' in self.mirna.columns:
            self.mirna.drop(['label'], axis=1, inplace=True)
        if 'case_id' in self.mirna.columns:
            self.mirna.drop(['case_id'], axis=1, inplace=True)
            
        if 'label' in self.rna.columns:
            self.rna.drop(['label'], axis=1, inplace=True)
        if 'case_id' in self.rna.columns:
            self.rna.drop(['case_id'], axis=1, inplace=True)
            
        if 'label' in self.illumina.columns:
            self.illumina.drop(['label'], axis=1, inplace=True)
        if 'case_id' in self.illumina.columns:
            self.illumina.drop(['case_id'], axis=1, inplace=True)
            
        self.mirna = pd.DataFrame(scaler.fit_transform(self.mirna))
        self.rna = pd.DataFrame(scaler.fit_transform(self.rna))
        self.illumina = pd.DataFrame(scaler.fit_transform(self.illumina))

        return


    def find_k_neighbors(self, row, i, k=None): 
        row=deepcopy(row)
        #case of P matrix
        if k==None:
            del row[i]  #delete element of the same column of row index
            return row

        #case of S (find k elements with minimum distance value of W[i][j])
        else:
            k_neighbors_index=[]
            neigh = 0
            max_value = max(row)
            for j in range(0, len(row)):
                if j!=i:
                    min_index = row.index(min(row))
                    k_neighbors_index.append(min_index)
                    neigh+=1
                    row[min_index] = max_value
                    if neigh == k:
                        return k_neighbors_index


    def P_matrix(self, W, n_case_id, name, save_matrix=False):
        if 'pStarting_matrix_'+name+'.pkl' in os.listdir('.'):
            print('Reading the file pickle for the p starting matrix {}'.format(name))
            df_p = pd.read_pickle('pStarting_matrix_'+name+'.pkl')
            return df_p.to_numpy()
            
        print('Calculating P matrix for {}...'.format(name))
        P=[]
        for i in tqdm(range(0, n_case_id)):
            row=[]
            for j in range(0,n_case_id):
                if i==j:
                    row.append(1/2)

                else:
                    k_neighbors = self.find_k_neighbors(W[i], i)
                    denominator = 2*sum(k_neighbors)
                    row.append(W[i][j]/denominator)
            P.append(row)
        #print(np.array(P))
        
        if save_matrix:
            df_P = pd.DataFrame(np.array(P))
            df_P.to_pickle('pStarting_matrix_'+name+'.pkl')
        return np.array(P)

    def S_matrix(self, W, n_case_id, name):
        print('Calculating S matrix for {}...'.format(name))
        S=[]
        for i in tqdm(range(0, n_case_id)):
            S_row=[]
            neighbors_indeces = self.find_k_neighbors(self.dict_dist.iloc[i,:].to_numpy().tolist(), i, self.k)
            for j in range(0,n_case_id):
                if j not in neighbors_indeces:
                    S_row.append(0)

                else:
                    np_row = np.array(W[i])
                    denominator = sum(np_row[neighbors_indeces])
                    S_row.append(W[i][j]/denominator)
            S.append(S_row)
        print(np.array(S))
        return np.array(S)
    
    def product_matrix(self, S_matrix, P_matrix):
        result = np.dot(S_matrix, P_matrix)
        result = np.dot(result, S_matrix.T)
        return result
    
    def sum_matrix_P(self, P1, P2):
        return np.add(P1,P2)/2
    
    def fit(self, num_iter=None):
        if num_iter is not None:
            self.p_rna = self.starting_p_rna.copy()
            self.p_mirna = self.starting_p_mirna.copy()
            self.p_illumina = self.starting_p_illumina.copy()
            for i in range(0, num_iter):
                self.p_rna_t1 = self.product_matrix(self.s_rna, self.sum_matrix_P(self.p_mirna, self.p_illumina))
                self.p_mirna_t1 = self.product_matrix(self.s_mirna, self.sum_matrix_P(self.p_rna, self.p_illumina))
                self.p_illumina_t1 = self.product_matrix(self.s_illumina, self.sum_matrix_P(self.p_mirna, self.p_rna))
                #print(self.p_rna_t1)
                #print(self.p_mirna_t1)
                #print(self.p_illumina_t1)
                self.p_rna = self.p_rna_t1
                self.p_mirna = self.p_mirna_t1
                self.p_illumina = self.p_illumina_t1
        else:
            print('ciao')

        return self
    
   
    def iterations_fit(self, matrices_diff=None, max_iter=100):
        if matrices_diff is not None:
            self.p_rna = self.starting_p_rna.copy()
            self.p_mirna = self.starting_p_mirna.copy()
            self.p_illumina = self.starting_p_illumina.copy()
            
            for step in range(0, max_iter):
                self.p_rna_t1 = self.product_matrix(self.s_rna, self.sum_matrix_P(self.p_mirna, self.p_illumina))
                self.p_mirna_t1 = self.product_matrix(self.s_mirna, self.sum_matrix_P(self.p_rna, self.p_illumina))
                self.p_illumina_t1 = self.product_matrix(self.s_illumina, self.sum_matrix_P(self.p_mirna, self.p_rna))
               
                self.p_rna = self.p_rna_t1
                self.p_mirna = self.p_mirna_t1
                self.p_illumina = self.p_illumina_t1

                diff_matrix = 0
                for i in range(0, len(self.p_rna)):
                    for j in range(i, len(self.p_rna)):
                        diff_matrix += np.abs(self.p_rna[i][j] - self.p_mirna[i][j])
                        diff_matrix += np.abs(self.p_illumina[i][j] - self.p_mirna[i][j])
                        diff_matrix += np.abs(self.p_illumina[i][j] - self.p_rna[i][j])
                
                diff_matrix = diff_matrix**0.5
                print(step, ':', diff_matrix)
                
                #diff_matrix = np.abs(np.subtract(self.p_rna, self.p_mirna)) + np.abs(np.subtract(self.p_rna, self.p_illumina)) + np.abs(np.subtract(self.p_mirna, self.p_illumina))
                #diff_matrix= np.abs(np.mean(diff_matrix))
                if diff_matrix<=np.abs(matrices_diff):
                    print('number of iterations to reach difference: ', step)
                    break
                    
                if step == max_iter-1: ##impossible to reach matrices difference
                    print('impossible to reach indicated difference, try with a bigger difference value')
        else:
            print('no difference for matrices found')

        return self
    
    def local_minimum_fit(self, iters_to_min=None, max_iter=100):
        if iters_to_min is not None:
            self.p_rna = self.starting_p_rna.copy()
            self.p_mirna = self.starting_p_mirna.copy()
            self.p_illumina = self.starting_p_illumina.copy()
            count=0
            prev_diff=0
            for step in range(0, max_iter):
                self.p_rna_t1 = self.product_matrix(self.s_rna, self.sum_matrix_P(self.p_mirna, self.p_illumina))
                self.p_mirna_t1 = self.product_matrix(self.s_mirna, self.sum_matrix_P(self.p_rna, self.p_illumina))
                self.p_illumina_t1 = self.product_matrix(self.s_illumina, self.sum_matrix_P(self.p_mirna, self.p_rna))
               
                self.p_rna = self.p_rna_t1
                self.p_mirna = self.p_mirna_t1
                self.p_illumina = self.p_illumina_t1

                diff_matrix = 0
                for i in range(0, len(self.p_rna)):
                    for j in range(i, len(self.p_rna)):
                        diff_matrix += np.abs(self.p_rna[i][j] - self.p_mirna[i][j])
                        diff_matrix += np.abs(self.p_illumina[i][j] - self.p_mirna[i][j])
                        diff_matrix += np.abs(self.p_illumina[i][j] - self.p_rna[i][j])
                
                diff_matrix = diff_matrix**0.5
                print(step, ':', diff_matrix)
                
                #diff_matrix = np.abs(np.subtract(self.p_rna, self.p_mirna)) + np.abs(np.subtract(self.p_rna, self.p_illumina)) + np.abs(np.subtract(self.p_mirna, self.p_illumina))
                #diff_matrix= np.abs(np.mean(diff_matrix))
                
                #check if a local minimum is found
                if int(diff_matrix)==prev_diff:
                    count+=1
                    if count>=iters_to_min:
                        print('local minimum reached in ', step, 'iterations')
                        break
                else:
                    count=0
                    
                prev_diff = int(diff_matrix)
                    
                if step == max_iter-1: ##impossible to reach matrices difference
                    print('impossible to reach local minimum, matrices seem to not converge')
        else:
            print('no minimum iterations for matrices found')

        return self
    
    def clean(self):
        del self.p_rna
        del self.p_mirna
        del self.p_illumina
        
        del self.p_rna_t1
        del self.p_mirna_t1
        del self.p_illumina_t1
        
        
        del self.w_rna
        del self.w_mirna
        del self.w_illumina
        
        return self

In [5]:
df_mirna.sort_values(by='case_id', inplace=True)
df_rna.sort_values(by='case_id', inplace=True)
df_illumina.sort_values(by='case_id', inplace=True)

sm = SimilarityNetworkFusion(df_mirna.drop_duplicates(subset='case_id'),
                            df_rna.drop_duplicates(subset='case_id'),
                            df_illumina.drop_duplicates(subset='case_id'), k=100).calculate_matrix()

  0%|          | 0/449 [00:00<?, ?it/s]

Calculating weights for RNA...


  0%|          | 0/449 [00:00<?, ?it/s]

Calculating weights for miRNA...


100%|██████████| 449/449 [01:20<00:00,  5.57it/s]


...

[[0.14703868 0.         0.         ... 0.         0.         0.        ]
 [0.         0.14176749 0.         ... 0.         0.         0.        ]
 [0.         0.         0.4847453  ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.45310793 0.         0.        ]
 [0.         0.00073598 0.         ... 0.         0.71467716 0.        ]
 [0.00958855 0.         0.         ... 0.         0.         0.11987761]]
Calculating S matrix for miRNA...


  2%|▏         | 9/449 [00:00<00:05, 87.39it/s]

[[9.51884388e-01 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 9.91371871e-01 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 8.82177133e-01 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 8.24222340e-01
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 3.84291372e-06 0.00000000e+00 ... 0.00000000e+00
  9.99269132e-01 0.00000000e+00]
 [4.45618677e-04 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 7.60057241e-01]]
Calculating S matrix for Illumina...


100%|██████████| 449/449 [00:05<00:00, 77.14it/s]


[[1.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.69620492e-35 0.00000000e+00 ... 0.00000000e+00
  1.00000000e+00 0.00000000e+00]
 [1.62184284e-53 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 1.00000000e+00]]


In [ ]:
sm.fit(num_iter=50)

In [69]:
sm.iterations_fit(matrices_diff=3)

0 : 25.458022271524985
1 : 18.538540093662142
2 : 15.449079255925637
3 : 13.040851847775372
4 : 11.486120522735616
5 : 10.339639092434838
6 : 9.45100130358968
7 : 8.749497080843888
8 : 8.16227107508217
9 : 7.670003341900004
10 : 7.244295946758629
11 : 6.874818324576108
12 : 6.549116177931862
13 : 6.260914033231143
14 : 6.002703030990444
15 : 5.770312722182554
16 : 5.559473999402204
17 : 5.367232570831043
18 : 5.190928998134345
19 : 5.028245975850384
20 : 4.877713250496675
21 : 4.737955701826748
22 : 4.607604904047421
23 : 4.485647069428842
24 : 4.371206106523313
25 : 4.263568525161656
26 : 4.1620854097189905
27 : 4.066247927113306
28 : 3.9755701774315675
29 : 3.889558430839133
30 : 3.8078023816436493
31 : 3.729945709186525
32 : 3.6557267378164453
33 : 3.5848444964258666
34 : 3.5170379328867245
35 : 3.4521122871750065
36 : 3.3898691533678633
37 : 3.3301854814553202
38 : 3.272870150395989
39 : 3.217742961983885
40 : 3.1647027228636793
41 : 3.113615523854089
42 : 3.0643709609991494
43 : 3

In [6]:
sm.local_minimum_fit(iters_to_min=8)

0 : 31.316604788291937
1 : 19.581563779946503
2 : 15.698067529237102
3 : 11.887030879754507
4 : 9.858604441593494
5 : 8.36262246583232
6 : 7.305448549410003
7 : 6.637090661919051
8 : 6.1149390464582964
9 : 5.7792866600348916
10 : 5.504176926681836
11 : 5.307419534957669
12 : 5.141144938590253
13 : 5.009428415269272
14 : 4.896230239719472
15 : 4.799644842770453
16 : 4.712350756257479
17 : 4.6317870215198305
18 : 4.555345326724179
19 : 4.481558882027976
20 : 4.409065115832527
21 : 4.337473094706606
22 : 4.2664323148673535
local minimum reached in  22 iterations


In [ ]:
import plotly.graph_objects as go

def plot_starting_graphs(df, y_pred):
    G = nx.from_numpy_matrix(df)
    print(nx.info(G))
    nx.draw_networkx(G, with_labels=False)
    return

In [ ]:
plot_starting_graphs(sm.p_rna, y_pred)

# Loading the label

In [7]:
y_illumina = LabelEncoder().fit_transform(df_illumina.drop_duplicates(subset='case_id').loc[:, 'label'].transform(lambda x: str(x)))
y_mirna = LabelEncoder().fit_transform(df_mirna.drop_duplicates(subset='case_id').loc[:, 'label'].transform(lambda x:  str(x)))
y_rna = LabelEncoder().fit_transform(df_rna.drop_duplicates(subset='case_id').loc[:, 'label'].transform(lambda x: str(x)))

In [8]:
y_pred = SpectralClustering(n_clusters=3, affinity='precomputed').fit(sm.p_mirna).labels_

/Users/carlozoccoli/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:73: UserWarning: Array is not symmetric, and will be converted to symmetric by average with its transpose.
  return f(**kwargs)


In [9]:
print('Rand Score:')
print('\tIllumina', adjusted_rand_score(y_illumina, y_pred))
print('\tMirna', adjusted_rand_score(y_mirna, y_pred))
print('\tRNA:', adjusted_rand_score(y_rna, y_pred))
print('\tmean:', (adjusted_rand_score(y_rna, y_pred) + adjusted_rand_score(y_illumina, y_pred) + adjusted_rand_score(y_mirna, y_pred))/3)

print('\n')
print('Silhouette score:')
print('\tIllumina', silhouette_score(sm.p_illumina, y_pred))
print('\tMirna', silhouette_score(sm.p_mirna, y_pred))
print('\tRNA:', silhouette_score(sm.p_rna, y_pred))

Rand Score:
	Illumina 0.7669745272440923
	Mirna 0.7626035617242862
	RNA: 0.7746477057016694
	mean: 0.7680752648900159


Silhouette score:
	Illumina 0.8959583489566215
	Mirna 0.8929830872918113
	RNA: 0.8999641267725116


In [ ]:
y_pred = KMeans(n_clusters=3).fit(sm.p_mirna).labels_
print('Rand Score:')
print('\tIllumina', adjusted_rand_score(y_illumina, y_pred))
print('\tMirna', adjusted_rand_score(y_mirna, y_pred))
print('\tRNA:', adjusted_rand_score(y_rna, y_pred))
print('\n')
print('Silhouette score:')
print('\tIllumina', silhouette_score(sm.p_illumina, y_pred))
print('\tMirna', silhouette_score(sm.p_mirna, y_pred))
print('\tRNA:', silhouette_score(sm.p_rna, y_pred))

In [ ]:
#####SI POTREBBE UTILIZZARE COME CONDIZIONE DI TERMINAZIONE CON UN CERTO VALORE
diff_matrix = np.abs(np.subtract(sm.p_rna, sm.p_mirna), sm.p_illumina)
print(np.mean(diff_matrix))